### Equity Earnings Projections
- 

In [1]:
# http://www.blackarbs.com/blog/get-free-financial-data-w-python-earnings-estimates-from-yahoo-finance/12/20/2015

def _get_eps_estimates(url):
    try:
        html_source = u.request.urlopen(url).read()
        soup = bs(html_source, 'lxml')
        # 
        # table
        table = soup.find_all('table', attrs={'class': 'yfnc_tableout1'})
        header = [th.text for th in table[0].find_all(class_='yfnc_tablehead1')]
        header_title = header[0]
        header_cols = header[1:5]
        index_row_labels = header[-5:]
        body = [[td.text for td in row.select('td')] for row in table[0].find_all('tr')]
        body = body[1:]
        df = pd.DataFrame.from_records(body)
        df = df.ix[:, 1:]
        df.index = index_row_labels
        header_cols = pd.Series(header_cols)
        header_cols = header_cols.str.replace(
            'Year', 'Year ').str.replace('Qtr.', 'Qtr. ')
        df.columns = header_cols
        eps_est = df.convert_objects(convert_numeric=True)
    except Exception as e:
        print(e)
    return eps_est

In [2]:
symbol = 'goog'

base_url = r'http://finance.yahoo.com/q/ae?s={}+Analyst+Estimates'.format(symbol)
eps_est = _get_eps_estimates(base_url)
eps_est

name 'u' is not defined


UnboundLocalError: local variable 'eps_est' referenced before assignment

In [3]:
# https://www.ryansmccoy.com/2019/01/28/download-eps-sales-estimates-using-python/

import pandas as pd
from datetime import datetime

df_calendar = pd.read_html(r'https://finance.yahoo.com/calendar/earnings/')

df_all_est = []

for ticker in df_calendar[0]['Symbol'].tolist():
    try:
        print(f'Downloading Estimates for {ticker}')

        df_est = pd.read_html(r'https://finance.yahoo.com/quote/{ticker}/analysis?p={ticker}')
        eps_est, rev_est = df_est[0], df_est[1]

        rev_est = rev_est.melt(id_vars=['Revenue Estimate'],var_name='Period')
        rev_est['Ticker'] = ticker
        rev_est['Item'] = "REVENUES"
        rev_est.rename(index=str, columns={"Revenue Estimate": "Statistic"}, inplace=True)
        rev_est['DateTime'] = str(datetime.utcnow())
        df_all_est.append(rev_est)

        eps_est = eps_est.melt(id_vars=['Earnings Estimate'],var_name='Period')
        eps_est['Ticker'] = ticker
        eps_est['Item'] = "EPS"
        eps_est.rename(index=str, columns={"Earnings Estimate": "Statistic"}, inplace=True)
        eps_est['DateTime'] = str(datetime.utcnow())
        df_all_est.append(eps_est)

        print(eps_est.append(rev_est).to_string())

    except Exception as e:
        print(f"Error Downloading {ticker} \n{e}\n")

df_all_merged = pd.concat(df_all_est)
df_all_merged.reset_index(drop=True, inplace=True)
df_all_merged = df_all_merged.T.reindex(['DateTime', 'Ticker', 'Item', 'Statistic', 'Period', 'value']).T

df_all_merged.to_csv(r'{}_estimates.csv'.format(datetime.today().strftime("%Y%M%d")))

ValueError: No tables found